In [30]:
path2target = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/n2p2/SiO2/02/run2"

In [31]:
import os
from typing import List, Any
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def remove_blank_from_array(arr: List[Any]) -> List[Any]:
    return list(filter(None, arr))

with open(os.path.join(path2target, "function.data"), "r") as f:
    lines = [s.strip() for s in f.readlines()]

In [32]:
idx_list = []
for i, line in enumerate(lines):
    if len(remove_blank_from_array(line.split(' '))) == 1:
        idx_list.append(i)

sf_val_list = []
for i, idx in enumerate(idx_list):
    try:
        block = lines[idx+1:idx_list[i+1]-1]
    except:
        block = lines[idx+1:-1]
    
    for l in block:
        sf_val_list.append(list(map(float, remove_blank_from_array(l.split(' ')))))

In [37]:
# 1つ目の要素はintなのでintに変換し、要素ごとに分ける
sf_val_list_dict = {
    "Si": [],
    "O": []
}
for sf_val in sf_val_list:
    if int(sf_val[0]) == 14:
        sf_val_list_dict["Si"].append(sf_val[1:])
    elif int(sf_val[0]) == 8:
        sf_val_list_dict["O"].append(sf_val[1:])
    else:
        raise ValueError("Unknown atomic number")

In [40]:
len(sf_val_list_dict["Si"]), len(sf_val_list_dict["O"]), len(sf_val_list_dict["O"]) / len(sf_val_list_dict["Si"])

(263392, 526784, 2.0)

# カラムをscalingのlogから取得する

In [65]:
number_of_sf_per_atom = 38

with open(os.path.join(path2target, 'nnp-scaling.log.0000'), mode='r') as f:
    scaling_log_lines = [s.strip() for s in f.readlines()]

keyword = "Short range atomic symmetry functions element  O :"
for line in scaling_log_lines:
    if keyword in line:
        keyword_idx = scaling_log_lines.index(line)


sf_info_list_raw = scaling_log_lines[keyword_idx+4: keyword_idx+4+number_of_sf_per_atom]
line_number_in_setting_file_list = [int(sf_info.split(' ')[-1]) for sf_info in sf_info_list_raw]

In [71]:
with open(os.path.join(path2target, 'input.nn'), mode='r') as f:
    lines = [s.strip() for s in f.readlines()]

target_sf_lines = [lines[idx-1] for idx in line_number_in_setting_file_list]

In [74]:
sf_columns = ["_".join(target_sf_line.split(' ')[1:]) for target_sf_line in target_sf_lines]

In [76]:
sf_df = pd.DataFrame(sf_val_list_dict["O"], columns=sf_columns)

# まとめて関数化する

In [87]:
from typing import Dict

def symmetry_function_values_dict(path2target: str, atom_num_symbol_map: Dict[int, str]) -> Dict[str, List[float]]:
    if not os.path.exists(os.path.join(path2target, "function.data")):
        raise ValueError("function.data does not exist")
    
    with open(os.path.join(path2target, "function.data"), "r") as f:
        lines = [s.strip() for s in f.readlines()]
    

    idx_list = []
    for i, line in enumerate(lines):
        if len(remove_blank_from_array(line.split(' '))) == 1:
            idx_list.append(i)

    sf_val_list = []
    for i, idx in enumerate(idx_list):
        try:
            block = lines[idx+1:idx_list[i+1]-1]
        except:
            block = lines[idx+1:-1]
        
        for l in block:
            sf_val_list.append(list(map(float, remove_blank_from_array(l.split(' ')))))
    
    # 初期化
    sf_val_list_dict = {}
    for _, atom_symbol in atom_num_symbol_map.items():
        sf_val_list_dict[atom_symbol] = []
    
    # 1つ目の要素はintなのでintに変換し、要素ごとに分ける
    for sf_val in sf_val_list:
        if int(sf_val[0]) in atom_num_symbol_map.keys():
            sf_val_list_dict[atom_num_symbol_map[int(sf_val[0])]].append(sf_val[1:])
        else:
            raise ValueError("Unknown atomic number please check atom_num_symbol_map")
    return sf_val_list_dict

In [103]:
def get_symmetry_function_columns(path2target: str, number_of_sf_per_atom: int, target_atom_symbol: str) -> List[str]:
    if not os.path.exists(os.path.join(path2target, 'nnp-scaling.log.0000')):
        raise ValueError("nnp-scaling.log.0000 does not exist")

    if not os.path.exists(os.path.join(path2target, 'input.nn')):
        raise ValueError("input.nn does not exist")
    
    with open(os.path.join(path2target, 'nnp-scaling.log.0000'), mode='r') as f:
        scaling_log_lines = [s.strip() for s in f.readlines()]

    keyword = f"Short range atomic symmetry functions element"
    keyword_idx = None
    for line in scaling_log_lines:
        if keyword not in line:
            continue
            
        atom_symbol_in_line = remove_blank_from_array(line.split(' '))[-2]
        if target_atom_symbol == atom_symbol_in_line:
            keyword_idx = scaling_log_lines.index(line)
    if keyword_idx is None:
        raise ValueError(f"{target_atom_symbol} does not exist")
    
    sf_info_list_raw = scaling_log_lines[keyword_idx+4: keyword_idx+4+number_of_sf_per_atom]
    line_number_in_setting_file_list = [int(sf_info.split(' ')[-1]) for sf_info in sf_info_list_raw]
    
    with open(os.path.join(path2target, 'input.nn'), mode='r') as f:
        lines = [s.strip() for s in f.readlines()]

        target_sf_lines = [lines[idx-1] for idx in line_number_in_setting_file_list]

    sf_info_list_raw = scaling_log_lines[keyword_idx+4: keyword_idx+4+number_of_sf_per_atom]
    line_number_in_setting_file_list = [int(sf_info.split(' ')[-1]) for sf_info in sf_info_list_raw]
    sf_columns = ["_".join(target_sf_line.split(' ')[1:]) for target_sf_line in target_sf_lines]
    return sf_columns

In [90]:
atom_num_symbol_map = {
    14: "Si",
    8: "O"
}

sf_val_list_dict = symmetry_function_values_dict(path2target, atom_num_symbol_map)

In [104]:
oxigen_columns = get_symmetry_function_columns(
    path2target=path2target,
    number_of_sf_per_atom=38,
    target_atom_symbol="O"
)

silicon_columns = get_symmetry_function_columns(
    path2target=path2target,
    number_of_sf_per_atom=38,
    target_atom_symbol="Si"
)

In [105]:
sf_silicon_df = pd.DataFrame(sf_val_list_dict["Si"], columns=silicon_columns)
sf_oxigen_df = pd.DataFrame(sf_val_list_dict["O"], columns=oxigen_columns)

In [1]:
from mlptools.analyzer.symmetry_function import SymmetryFunctionValueReader

path2target = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/n2p2/SiO2/02/run2"
atom_num_symbol_map = {
    14: "Si",
    8: "O"
}

sf_reader = SymmetryFunctionValueReader(path2target)
sf_df_dict = sf_reader.read(
    atom_num_symbol_map=atom_num_symbol_map,
    number_of_sf_per_atom=38
)

All files exist
Si symmetry function columns are created
Symmetry function dataframe of Si is created
O symmetry function columns are created
Symmetry function dataframe of O is created


In [2]:
sf_df_dict["Si"]

,Si_2_O_0.02778_0.0_6.0,Si_2_Si_0.02778_0.0_6.0,Si_2_O_0.05048_0.0_6.0,Si_2_Si_0.05048_0.0_6.0,Si_2_O_0.09172_0.0_6.0,Si_2_Si_0.09172_0.0_6.0,Si_2_O_0.1667_0.0_6.0,Si_2_Si_0.1667_0.0_6.0,Si_2_O_0.3029_0.0_6.0,Si_2_Si_0.3029_0.0_6.0,...,Si_3_Si_Si_0.05556_-1.0_1.0_6.0_0.0,Si_3_O_O_0.05556_1.0_1.0_6.0_0.0,Si_3_Si_O_0.05556_1.0_1.0_6.0_0.0,Si_3_Si_Si_0.05556_1.0_1.0_6.0_0.0,Si_3_O_O_0.05556_-1.0_6.0_6.0_0.0,Si_3_Si_O_0.05556_-1.0_6.0_6.0_0.0,Si_3_Si_Si_0.05556_-1.0_6.0_6.0_0.0,Si_3_O_O_0.05556_1.0_6.0_6.0_0.0,Si_3_Si_O_0.05556_1.0_6.0_6.0_0.0,Si_3_Si_Si_0.05556_1.0_6.0_6.0_0.0
0,2.061411,0.676837,1.768714,0.539055,1.396036,0.369609,1.005106,0.202730,0.666532,0.078381,...,0.000953,0.093849,0.090054,0.002430,0.010891,0.002436,0.000098,0.032292,0.065918,0.001252
1,2.086123,0.659339,1.792848,0.522207,1.421538,0.354173,1.035684,0.190168,0.704829,0.070576,...,0.000814,0.095198,0.086892,0.002231,0.012954,0.002362,0.000075,0.033412,0.064807,0.001145
2,2.007661,0.685782,1.719912,0.547357,1.357265,0.377007,0.983126,0.208599,0.663879,0.081695,...,0.001037,0.085995,0.089647,0.002449,0.010795,0.002578,0.000109,0.029260,0.064298,0.001231
3,2.015299,0.670310,1.728042,0.532230,1.365905,0.362760,0.991903,0.196558,0.671846,0.073979,...,0.000891,0.087004,0.087144,0.002295,0.011469,0.002214,0.000085,0.030103,0.065167,0.001152
4,1.385808,0.366701,1.202968,0.290496,0.966034,0.195077,0.705890,0.099412,0.457098,0.031374,...,0.000080,0.035360,0.033247,0.000203,0.004991,0.000620,0.000009,0.009037,0.025172,0.000108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263387,3.355732,1.307483,2.849588,1.055140,2.201889,0.737308,1.528329,0.418398,0.983426,0.179477,...,0.007212,0.271679,0.263363,0.018714,0.023608,0.008167,0.000522,0.102349,0.172356,0.008373
263388,3.193920,1.243746,2.705138,1.000568,2.083393,0.694968,1.443627,0.389669,0.932225,0.163168,...,0.006027,0.236262,0.232100,0.016171,0.021194,0.007218,0.000442,0.087037,0.150142,0.007269
263389,3.207149,1.251575,2.717103,1.009643,2.089749,0.705251,1.436390,0.399951,0.906221,0.170833,...,0.006444,0.242160,0.242448,0.016538,0.020066,0.007236,0.000496,0.095376,0.160541,0.007342
263390,3.210754,1.239432,2.722288,0.996764,2.098010,0.692092,1.449683,0.388008,0.924570,0.162429,...,0.005938,0.244188,0.236179,0.016028,0.021054,0.007033,0.000435,0.094888,0.155479,0.007263
